## Итоговое задание
Вы получили данные от интернет-магазина, который продает товары онлайн. Вашей задачей является анализ данных, выявление инсайтов и предоставление рекомендаций для улучшения показателей магазина.

#### Данные

База данных (SQL):

__Orders__ — информация о заказах (ID заказа, ID клиента, дата заказа, сумма заказа, способ оплаты).  
__Customers__ — информация о клиентах (ID клиента, возраст, пол, город).  
__Products__ — информация о товарах (ID товара, название товара, категория, цена).  
__OrderItems__ — информация о товарах в заказах (ID заказа, ID товара, количество).  
__Visits__ — данные о посещениях сайта (ID клиента, дата визита, источник трафика, версия сайта).  
-- версия сайта используется только для проведения АВ теста

CSV файлы:

__Promotions.csv__ — данные о маркетинговых акциях (ID акции, ID товара, дата начала акции, дата окончания акции).  
__Weather.csv__ — данные о погоде (дата, город, температура, осадки) для анализа влияния погодных условий на продажи.


### Задания

#### Исследование данных:

Проведите первичный анализ всех доступных данных (SQL, CSV).  
Проверьте наличие пропусков и дубликатов. Опишите структуру данных, выделите ключевые переменные.


#### Обогащение данных:

- Загрузите данные о маркетинговых акциях из CSV файла и присоедините их к основным данным.  
- Извлеките данные о погоде для городов, представленных в базе данных. Присоедините их к данным о заказах по дате и городу.


#### Анализ продаж:

- Определите влияние маркетинговых акций на продажи. Какие товары лучше продавались во время акций?  
- Выявите, как погодные условия влияли на количество заказов в разных городах. Например, увеличивались ли продажи в дождливые дни?


#### Сегментация клиентов:

- Проведите сегментацию клиентов по частоте покупок и сумме потраченных средств. Определите, какие сегменты наиболее ценны для бизнеса.  
- Проанализируйте влияние акций на поведение клиентов из разных сегментов.


#### Анализ трафика:

Проанализируйте, какие источники трафика приводят наиболее платежеспособных клиентов. Влияют ли маркетинговые акции на привлечение клиентов через определенные каналы?


#### Поведение клиентов:

- Найдите среднее количество дней между заказами для активных клиентов. Как меняется этот показатель в зависимости от участия в акциях?
- Проведите анализ по возрастным группам: какие товары и категории чаще покупают разные возрастные категории? Как на это влияют акции?


#### Визуализация данных:

Постройте графики, которые визуализируют директору ключевые метрики: динамику продаж по месяцам, влияние погоды и акций на продажи, распределение возрастов клиентов и т.д. Покажите все, что по вашему мнению, важно подсветить директору для принятия управленческих решений.


#### Продуктовые метрики:

Рассчитайте основные продуктовые метрики: DAU, CAC (стоимость привлечения клиента), LTV (пожизненная ценность клиента).


#### A/B тестирование:

Компания тестирует две разные версии главной страницы своего сайта — версию A (стандартная страница) и версию B (страница с обновленным дизайном, где более активно представлены рекламные акции).  
Для каждой версии страницы собираются данные о визитах пользователей и их конверсии в заказы. Ваша задача — провести A/B тест и определить, есть ли статистически значимая разница в конверсии между этими двумя версиями.


#### Рекомендации:

На основе анализа данных, предложите три ключевые рекомендации по улучшению показателей магазина, учитывая влияние акций и внешних факторов (например, погоды).


### Дополнительно:  

Все расчеты должны сопровождаться пояснениями.  
Используйте SQL для извлечения данных, Python для обработки, обогащения и визуализации, статистические методы для проведения A/B тестов.

In [1]:
# Пример подключение к базе данных

import sqlite3
import pandas as pd

# Подключение к базе данных
conn = sqlite3.connect('ecommerce_data.db')

# Создание курсора для выполнения SQL-запросов
cursor = conn.cursor()

# Пример SQL-запроса для получения всех заказов
# query = '''
# SELECT * FROM orders
# LIMIT 10;
# '''

# Выполнение запроса и получение данных в DataFrame
# orders_df = pd.read_sql_query(query, conn)

# # Показать первые строки результата
# print(orders_df.head())

# # Выполнение более сложного запроса (например, соединение двух таблиц)
# query = '''
# SELECT o.order_id, o.order_date, c.city, c.gender, o.order_amount
# FROM orders o
# JOIN customers c ON o.customer_id = c.customer_id
# WHERE c.city = 'Moscow'
# AND o.order_amount > 1000
# ORDER BY o.order_date DESC
# LIMIT 10;
# '''

# # Получение результата
# result_df = pd.read_sql_query(query, conn)

# # Показать результат запроса
# print(result_df)

# Закрытие подключения к базе данных
# conn.close()

In [4]:
df_prom = pd.read_csv('Promotions.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Promotions.csv'

Исследование данных:

Проведите первичный анализ всех доступных данных (SQL, CSV).  

Проверьте наличие пропусков и дубликатов. Опишите структуру данных, выделите ключевые переменные.

In [4]:
# Получаем информацию о имеющихся таблицах
query = '''
SELECT name FROM sqlite_master WHERE type='table';
'''
tables_df = pd.read_sql_query(query, conn)
for i, row in tables_df.iterrows():
    query = f'''
    SELECT * FROM {row['name']}
    LIMIT 2;
    '''
    result_df = pd.read_sql_query(query, conn)
    print(f"Название таблицы: {row['name']}")
    print(result_df)

Название таблицы: customers
   customer_id  age gender              city
0            1   56   Male            Moscow
1            2   69   Male  Saint Petersburg
Название таблицы: products
   product_id product_name category   price
0           1      Current    Books  969.89
1           2     Majority     Home  777.38
Название таблицы: orders
   order_id  customer_id  order_date  order_amount payment_method
0         1          436  2024-07-03       1462.87    Credit Card
1         2           72  2023-10-20       4417.12     Debit Card
Название таблицы: order_items
   order_id  product_id  quantity
0         1          61  4.927976
1         2          24  5.404460
Название таблицы: visits
   visit_id  customer_id           visit_date traffic_source version  \
0         1          103  2024-07-02 00:00:00        AdWords       B   
1         2          436  2024-07-03 00:00:00   Social Media       B   

   visit_cost  
0         7.5  
1         4.5  
   COUNT(*)
0         0
   COUNT(

In [5]:
# Проверяем пропуски
query_list = ['SELECT COUNT(*) FROM orders WHERE customer_id IS NULL;',\
'SELECT COUNT(*) FROM customers WHERE age IS NULL OR city IS NULL;',\
'SELECT COUNT(*) FROM products WHERE product_name IS NULL OR category IS NULL;',\
'SELECT COUNT(*) FROM order_items WHERE order_id IS NULL OR product_id IS NULL;',\
'SELECT COUNT(*) FROM visits WHERE customer_id IS NULL OR visit_date IS NULL;']

for i in query_list:
    query = i
    result_df = pd.read_sql_query(query, conn)
    print(result_df)

   COUNT(*)
0         0
   COUNT(*)
0         0
   COUNT(*)
0         0
   COUNT(*)
0         0
   COUNT(*)
0         0


In [ ]:
# Проверяем дубликаты ###   проверка git 
query_list = ['SELECT customer_id, COUNT(*) FROM customers GROUP BY customer_id HAVING COUNT(*) > 1;',\
'SELECT order_id, product_id, SUM(quantity) FROM order_items GROUP BY order_id, product_id HAVING SUM(quantity) > 0;']

for i in query_list:
    query = i
    result_df = pd.read_sql_query(query, conn)
    print(result_df)

Empty DataFrame
Columns: [customer_id, COUNT(*)]
Index: []
      order_id  product_id  SUM(quantity)
0            1          61       4.927976
1            2          24       5.404460
2            3          38       5.329771
3            4          92       2.557975
4            5          55       4.798695
...        ...         ...            ...
3408      3409          74       5.228027
3409      3410          39       5.018066
3410      3411          86       1.931658
3411      3412           8       0.092211
3412      3413          65       5.004615

[3413 rows x 3 columns]
